### Compare DAV Wildschutzgebiete (epsg:4326 shape file) with mapped Wildschutzgebiete (XCTrails geojson)

In [1]:
import fiona
from shapely.geometry import shape, mapping
from shapely.ops import transform
from shapely.geometry import Polygon

import copy

cat = 'Wildschutzgebiet'

mappedFile = 'E:/OSM/Schutzgebiete/Schongebiete.geojson' # epsg:4326

shapeFile = 'E:/OSM/Schutzgebiete/200126_Schutzgebiete_By-Karten/Schutzgebiete_BY-Karten-epsg4326.shp' # epsg:4326
outFile = f'E:/OSM/Schutzgebiete/200126_Schutzgebiete_By-Karten/Schutzgebiete_BY-Karten-{cat}.shp' # epsg:4326

In [2]:
mapped_features = []
with fiona.open(mappedFile) as input:
    oldCrs = input.crs
    for feat in input:
        if feat['geometry'] != None and len(feat['geometry']['coordinates'][0]) > 2:
            mapped_features.append(feat)
size = len(mapped_features)
print(f"OSM-Gebiete in {mappedFile}: {size}")

OSM-Gebiete in E:/OSM/Schutzgebiete/Schongebiete.geojson: 557


In [3]:
# find a given poly in a list of geometry features and return its (mapped) properties
def find_poly(poly, features):
    poly = poly.buffer(0.0001)
    for f in features:
        p = Polygon(shape(f['geometry'])).buffer(0.0001)
        iou = poly.intersection(p).area / poly.union(p).area
        if iou > 0.995:
            return f['properties']
    return None

In [9]:
# read and 3D to 2D convert DAV shapefile
newFeatures = []
with fiona.open(shapeFile) as input:
    schema = input.schema
    crs = input.crs
    driver = input.driver
    for feat in input:
        if 'Kategorie' in feat['properties'] and feat['properties']['Kategorie'] != None and feat['properties']['Kategorie'] == cat:
            if feat['geometry'] != None:
                if len(feat['geometry']['coordinates']) > 1:
                    # multipolygons - code to be improved...
                    for pfeat in feat['geometry']['coordinates']:
                        try: # some are len(1) lists, some aren't ?!
                            poly = Polygon(pfeat[0])
                        except:
                            poly = Polygon(pfeat)
                        poly = transform(lambda x, y, z=None: (x, y), poly)
                        feat2 = copy.deepcopy(feat)
                        feat2['geometry'] = mapping(poly)
                        newFeatures.append(feat2)
                    continue
                if len(feat['geometry']['coordinates'][0]) < 3:
                    print("Skipping 2-point line")
                    continue
                # transform 3D to 2D
                poly = shape(feat['geometry'])
                poly = transform(lambda x, y, z=None: (x, y), poly)
                feat['geometry'] = mapping(poly)
                mapped_props = find_poly(poly, mapped_features)
                if not mapped_props:
                    newFeatures.append(feat)
                else:
                    # print(mapped_props)
                    # visual check in output:
                    # - type1 oder type2
                    # - name = Wildschutzgebiet ...
                    # - Regelung passt mit description und access:conditional zusammen
                    # WhiteList:
                    # - Wald-Wild-Schongebiet Bärenfalle 
                    print(f"{feat['properties']['Name']} {feat['properties']['Regelung']}:")
                    print(f"\t{mapped_props['@id']} {mapped_props['classification']} {mapped_props['name']} {mapped_props['description']} {mapped_props['access:conditional']}")

with fiona.open(outFile, 'w', crs={'init':'epsg:4326'}, driver='ESRI Shapefile', schema=schema) as out:
    for f in newFeatures:
        out.write(f)

size = len(newFeatures);          
print(f"Gebiete in {shapeFile}: {size}")

"Wirtsalm" Betretungsverbot: 01.11. bis 31.03.:
	way/923656164 type2 Wildschutzgebiet Wirtsalm Betretungsverbot: 01.11. bis 31.03. no @ (Nov - Mar)
"Feichteck" Betretungsverbot: 15.10. bis 15.05.:
	way/925219223 type2 Wildschutzgebiet Feichteck Betretungsverbot: 15.10. bis 15.05. no @ (Oct 15 - May 15)
"Grattenbach" Betretungsverbot: 01.11. bis 31.05.:
	way/925194626 type2 Wildschutzgebiet Grattenbach Betretungsverbot: 01.11. bis 31.05. no @ (Nov - May)
"Gründberg" Betretungsverbot: 01.11. bis 30.04.:
	way/925185305 type2 Wildschutzgebiet Gründberg Betretungsverbot: 01.11. bis 30.04. no @ (Nov 1 - Apr 30)
"Bärenfalle" Betretungsverbot: 15.01. bis 30.04.:
	way/923832592  Wald-Wild-Schongebiet Bärenfalle Wald-Wild-Schongebiet, mit der Bitte dieses nicht zu betreten oder zu befahren. None
"Halmrauth" Betretungsverbot: 01.11. bis 15.04.:
	way/925199679 type2 Wildschutzgebiet Halmrauth Betretungsverbot: 01.11. bis 15.04. no @ (Nov 1 - Apr 15)
"Bärenleiten" Betretungsverbot: 01.11. bis 15.04